In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [12]:
from public_data.models import Region
from django.contrib.gis.db.models.functions import Centroid
from django.contrib.gis.db.models import Union

result = Region.objects.annotate(mpoly_union=Union("mpoly")).values("mpoly_union").annotate(center=Centroid("mpoly_union"))
result = result.first()
result

{'mpoly_union': <Polygon object at 0x7f934e3b2310>,
 'center': <Point object at 0x7f934e3b2290>}

In [16]:
print(result["center"].x)
print(result["center"].y)

4.536186161788389
45.51461100292581


In [19]:
from project.models import Request
from django.db.models import Count, CharField, Func, F, Value

qs_dl = (
    Request.objects.annotate(
        date=Func(
            F("created_date"),
            Value("yyyy-MM"),
            function="to_char",
            output_field=CharField(),
        )
    )
    .values("date")
    .annotate(total=Count("id"))
    .order_by("date")
)
qs_dl

<QuerySet [{'date': '2022-06', 'total': 3}]>